# Hackthon_Solution

## Step 1 : Reading the Datasets

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## Step 2 : Missing Data Treatment

In [4]:
train.isna().sum().head(20)

id                0
gender            0
area              0
qualification     0
income            0
marital_status    0
vintage           0
claim_amount      0
num_policies      0
policy            0
type_of_policy    0
cltv              0
dtype: int64

In [5]:
test.isna().sum().head(20)

id                0
gender            0
area              0
qualification     0
income            0
marital_status    0
vintage           0
claim_amount      0
num_policies      0
policy            0
type_of_policy    0
dtype: int64

## Step 3 : Defining X and Y

In [6]:
Y = train[["cltv"]]
X = train.drop(labels=["cltv","id"],axis=1)

In [7]:
Y.head()

,cltv
0,64308
1,515400
2,64212
3,97920
4,59736


In [8]:
X.head(20)

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum
1,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum
2,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum
3,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum
4,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold
5,Male,Rural,High School,More than 10L,1,1,0,1,C,Gold
6,Female,Rural,Bachelor,5L-10L,1,6,0,More than 1,A,Gold
7,Female,Urban,High School,5L-10L,0,0,5473,More than 1,A,Silver
8,Female,Urban,High School,5L-10L,1,3,6105,More than 1,A,Gold
9,Male,Rural,High School,More than 10L,1,6,0,More than 1,A,Platinum


## 4. EDA(Exploratory Data Analysis)

Here we have both categorical as well as continuous coloums so we have to convert them in to continuous first we have to seperate them

### Separating Categorical and Continuous coloumns,Defining Preprocessing,Standardize,Anova,Chisquare and Outliers

In [9]:
def preprocessing(df):
    cat,con = catconsep(df)
    X1 = standardize(df)
    import pandas as pd
    X2 = pd.get_dummies(df[cat])
    Xnew = X1.join(X2)
    return Xnew

def standardize(df):
    import pandas as pd
    cat,con = catconsep(df)
    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X1 = pd.DataFrame(ss.fit_transform(df[con]),columns=con)
    return X1

def ANOVA(df,cat,con):
    from pandas import DataFrame
    from statsmodels.api import OLS
    from statsmodels.formula.api import ols
    rel = con + " ~ " + cat
    model = ols(rel,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    Q = DataFrame(anova_results)
    a = Q['PR(>F)'][cat]
    return round(a,3)

def chisquare(df,cat1,cat2):
    from scipy.stats import chi2_contingency
    import pandas as pd
    ct = pd.crosstab(df[cat1],df[cat2])
    a,b,c,d = chi2_contingency(ct)
    return b

def outliers(df):
    con = []
    for i in df.columns:
        if(df[i].dtypes != "object"):
            con.append(i)
    outliers = []
    for i in con:
        outliers.extend(list(df[df[i]>3].index))
        outliers.extend(list(df[df[i]<-3].index))
    from numpy import unique
    return list(unique(outliers))

def catconsep(df):
    cat = []
    con = []
    for i in df.columns:
        if(df[i].dtypes == "object"):
            cat.append(i)
        else:
            con.append(i)
    return cat,con

## Checking ANOVA values

In [10]:
cat = []
con = []
for i in X.columns: 
    if(X[i].dtypes=="object"):
        cat.append(i)
    else:
        con.append(i)

In [11]:
for i in con:
    print("-------------","cltv vs",i,"-------------")
    print(ANOVA(train,"cltv",i))

------------- cltv vs marital_status -------------
0.0
------------- cltv vs vintage -------------
0.0
------------- cltv vs claim_amount -------------
0.0


## Checking Chisquare Value

In [12]:
for i in cat:
    print("----------","cltv vs",i,"-----------")
    print(chisquare(train,"cltv",i))

---------- cltv vs gender -----------
0.024135469316112507
---------- cltv vs area -----------
1.0
---------- cltv vs qualification -----------
1.0
---------- cltv vs income -----------
2.0724262047274186e-49
---------- cltv vs num_policies -----------
0.0
---------- cltv vs policy -----------
1.8426826884024617e-35
---------- cltv vs type_of_policy -----------
0.3106478022706552


In [13]:
cat

['gender',
 'area',
 'qualification',
 'income',
 'num_policies',
 'policy',
 'type_of_policy']

In [14]:
con

['marital_status', 'vintage', 'claim_amount']

## Step 5 : Preprocessing of the Data

In [15]:
Xnew = preprocessing(X)

In [16]:
Xnew

,marital_status,vintage,claim_amount,gender_Female,gender_Male,area_Rural,area_Urban,qualification_Bachelor,qualification_High School,qualification_Others,...,income_<=2L,income_More than 10L,num_policies_1,num_policies_More than 1,policy_A,policy_B,policy_C,type_of_policy_Gold,type_of_policy_Platinum,type_of_policy_Silver
0,0.858869,0.176531,0.440940,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,1,0
1,-1.164322,1.486327,0.223305,0,1,1,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
2,0.858869,1.486327,-0.537192,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,1,0
3,-1.164322,1.049728,-1.333859,1,0,1,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
4,0.858869,0.613129,-0.258557,0,1,0,1,0,1,0,...,0,1,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89387,-1.164322,0.613129,-1.333859,0,1,1,0,0,1,0,...,0,1,0,1,0,1,0,0,1,0
89388,0.858869,-2.006463,-0.139318,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,1,0
89389,-1.164322,-0.260068,-0.303004,1,0,0,1,1,0,0,...,0,0,0,1,0,1,0,0,1,0
89390,-1.164322,1.486327,-1.333859,1,0,1,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0


## Step 6 : Training,Testing and Splitting Model

In [17]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.4,random_state=21)

## Step 7 : Decision Tree

Here We have used Decision tree as we have labelled data in Y and we want a good accuracy

In [18]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(criterion="mse",random_state=21,max_depth=5)

model = dtr.fit(xtrain,ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

In [19]:
from sklearn.metrics import r2_score
r2_score(ytest,pred_ts)

0.15247861791937145

## Random Forest Regressor

In [20]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(criterion="mse",random_state=21,max_depth=5)
model = rfr.fit(xtrain,ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

## R2_Score

In [21]:
from sklearn.metrics import r2_score
r2_score(ytest,pred_ts)

0.15440491862471084

## Step 8 : Getting Data for Prediction

In [22]:
B = pd.read_csv("test.csv")

In [23]:
B1 = B[X.columns].head()

In [24]:
B1

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,Female,Rural,High School,5L-10L,0,6,2134,More than 1,B,Silver
1,Female,Urban,High School,2L-5L,0,4,4102,More than 1,A,Platinum
2,Male,Rural,High School,5L-10L,1,7,2925,More than 1,B,Gold
3,Female,Rural,Bachelor,More than 10L,1,2,0,More than 1,B,Silver
4,Female,Urban,High School,2L-5L,0,5,14059,More than 1,B,Silver


In [25]:
B1.isna().sum()

gender            0
area              0
qualification     0
income            0
marital_status    0
vintage           0
claim_amount      0
num_policies      0
policy            0
type_of_policy    0
dtype: int64

In [26]:
cat = []
con = []
for i in B.columns:
    if(B[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [27]:
cat

['gender',
 'area',
 'qualification',
 'income',
 'num_policies',
 'policy',
 'type_of_policy']

In [28]:
con

['id', 'marital_status', 'vintage', 'claim_amount']

In [29]:
con = B.drop("id",axis=1)

In [30]:
B2 = con

In [31]:
B3 = pd.get_dummies(B[cat])

In [32]:
B4 = B2.join(B3)

In [33]:
Bnew = standardize(B4)

## Step 9 : Final Submission

In [34]:
ready_for_pred = Bnew[Xnew.columns]

In [35]:
predicted_final = model.predict(ready_for_pred)

In [36]:
final_submission = pd.DataFrame([B.id,predicted_final]).T
final_submission.columns = ["id","cltv"]

In [37]:
final_submission.to_csv("final_submission.csv",index=False)

In [38]:
final_submission.head(10)

,id,cltv
0,89393.0,92158.530903
1,89394.0,124793.293210
2,89395.0,96556.913113
3,89396.0,89581.908301
4,89397.0,138675.622416
5,89398.0,110406.513150
6,89399.0,47853.632897
7,89400.0,115318.241036
8,89401.0,47852.718270
9,89402.0,153000.370635


# ----------------------------------------------------------------------------------------